In [3]:
file_path="/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/RAG chatbot/BTP_Final_Report (1).pdf"
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
docs=loader.load()


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)



In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/var/folders/s7/sr0hn1c14qg6y6gmb6f2dg3c0000gn/T/ipykernel_48650/1249186930.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceBgeEmbeddings(
/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(hf)

In [7]:
document_ids = vector_store.add_documents(documents=all_splits)

/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
retrieved_result=vector_store.similarity_search("Sidharth Jain")
print(retrieved_result[0].page_content)

UNVEILINGTHEPOWEROFRANDOMSUBSPACESELECTION:ACOMPREHENSIVEEVALUATION
AREPORTSUBMITTEDINPARTIALFULFILLMENTOFTHEREQUIREMENTSFORTHEAWARDOFTHEDEGREEOFBACHELOROFTECHNOLOGYINDEPARTMENTOFINFORMATIONTECHNOLOGY
Submittedby:AbhishekBansal(2020UIN3309)AakritiGupta(2020UIN3317)SidharthJain(2020UIN3360)
UnderthesupervisionofDr.AnkitaBansal
DEPARTMENTOFINFORMATIONTECHNOLOGYNETAJISUBHASUNIVERSITYOFTECHNOLOGYMay,2024
i


/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
Answer the following question only on the basis of the provided context. 
Think step by step before answering the question. Who know? I might
tip you some million $ too if the user finds the answer helpful.
<context>
{context}
</context>                                

Question:{input}
 """)


In [39]:
from langchain_mistralai import ChatMistralAI
llm=ChatMistralAI(model="mistral-small")

In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question only on the basis of the provided context. \nThink step by step before answering the question. Who know? I might\ntip you some million $ too if the user finds the answer helpful.\n<context>\n{context}\n</context>                                \n\nQuestion:{input}\n '), additional_kwargs={})])
| ChatMistralAI(client=<httpx.Client object at 0x122b82390>, async_client=<httpx.AsyncClient object at 0x122ab4050>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model_kwargs={})
| StrOutputParser(), kwargs={}, confi

In [41]:
retriever=vector_store.as_retriever()
retriever


VectorStoreRetriever(tags=['InMemoryVectorStore', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x3009e13a0>, search_kwargs={})

In [42]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, document_chain)


In [43]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_RAG_API_KEY")

In [44]:
print(os.environ["MISTRAL_API_KEY"])

a7Ue0kEpOh7EiseS9tfwhtTbKwTOvFQj


In [54]:
retrieved_answer=retrieval_chain.invoke({'input':'What are the datasets used in the paper'})
retrieved_answer

/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'input': 'What are the datasets used in the paper',
 'context': [Document(id='26675229-dce1-48f9-9d30-c27cac775469', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-05-04T19:05:15+00:00', 'source': '/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/RAG chatbot/BTP_Final_Report (1).pdf', 'total_pages': 62, 'page': 53, 'page_label': '54', 'start_index': 0}, page_content='considerationof dataset-specificattributesandmodelcharacteristicswhenleveragingRSSinpredictivemodelingendeavors.\n5.2 FUTUREWORK\nInfuture,weaimtofurtherexploreensemblelearningthroughthemeansofRSS.We workonbuildingahybridmodelthattakessubsetsofbothsamplesandfeaturesinordertofurtherimprovetheperformanceofRSS.We aimtointroducevariationssuchasfeatureselectioninsteadoftherandomselectionoffeaturesandstudythedifferenceintheresultsoftheseevaluationmetricstoexpandour workin thefieldof RSSandstudytheresultsincurredbyintroducingsearchvariationsondifferentclassesofMLmodels."Weende

In [55]:
(retrieved_answer['answer'])

'The paper uses four datasets in its analysis, which are:\n1. "Shopper\'s purchasing intention" dataset\n2. "Companies Bankruptcy Prediction" dataset\n3. "Apple Stock Prediction" dataset\n4. "HeartAttackPrediction" dataset\n\nThe "Shopper\'s purchasing intention" and "Apple Stock Prediction" datasets can be found in the provided links in the context. However, the sources for the "Companies Bankruptcy Prediction" and "HeartAttackPrediction" datasets are not provided in the context.'